# Decorate your satellite images to make them ready for publication
This example demonstrate how you can add various overlays like coastlines, borders and graticules and decorate your image with text, points, logos, etc, in order to make it ready for publication.

In [1]:
import os
from glob import glob
from satpy import Scene

DATA_DIR = "/home/a000680/data/hrit/20200923"
FILES = glob(os.path.join(DATA_DIR, "*"))

areaid = 'euro4'
composite_name = 'overview'
font = "/usr/share/fonts/dejavu/DejaVuSerif.ttf"
pytroll_logo = "/home/a000680/data/logos/pytroll_dark_small.png"
eumetsat_logo = "/home/a000680/data/logos/eumetsat_logo.gif"


First we prepare a dictionary specifying how we want the graticules to be later drawn over the image.

In [2]:
grid = {'major_lonlat': (10, 10), 'minor_lonlat': (2, 2),
        'outline': (255, 255, 0), 'outline_opacity': 175,
        'width': 1.0, 'minor_width': 0.5, 'minor_is_tick': 1,
        'write_text': True, 'lat_placement': 'lr', 'lon_placement': 'b',
        'font': font}

We also want to point out a few geographical places, in this case four capitols in Europe.
Each city will be marked by a white dot with a black ring around, and for each dot the name of the city is written in a red text box:

In [3]:
poi_list = [((2.3522, 48.8566), 'Paris'),
            ((0.1278, 51.5074), 'London'),
            ((12.568, 55.676), 'Copenhagen'),
            ((24.938, 60.170), 'Helsinki')]
points = {'font': font,
          'font_size': 20, 'points_list': poi_list, 'symbol': 'circle', 'ptsize': 5,
          'outline': 'black', 'fill': 'white', 'width': 8, 'fill_opacity': 255, 'box_outline': 'black',
          'box_linewidth': 2.0, 'box_fill': (255, 0,  0), 'box_opacity': 127}

Now prepare the dictionaries with specifications for the coastlines, borders and rivers to be shown on the image:

In [5]:
coast = {'outline': (255, 255, 255), 'width': 1.5, 'level': 1, 'resolution': 'l'}
borders = {'outline': (255, 255, 255), 'width': 1.0, 'level': 3, 'resolution': 'i'}
rivers = {'outline': (0,   0, 255), 'width': 1.0, 'level': 3, 'resolution': 'i'}

Now load the Meteosat SEVIRI scene and resample it onto the area specified via its name above (euro4):

In [6]:
scn = Scene(filenames=FILES, reader='seviri_l1b_hrit')
scn.load([composite_name])
local_scn = scn.resample(areaid, radius_of_influence=35000.0)

/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()


Now we prepare how to decorate the image with logos and a header text including the time of observation, taken from the scene object:

In [7]:
start_time_txt = local_scn.start_time.strftime('%Y-%m-%d %H:%M')

decoration = [
    {'logo': {'logo_path': pytroll_logo,
              'height': 60, 'bg': 'white', 'bg_opacity': 120}},
    {'logo': {'logo_path': eumetsat_logo,
              'height': 60, 'bg': 'white', 'bg_opacity': 120}},
    {'text': {'txt': 'Meteosat-11' + ' ' + start_time_txt +
              "\nComposite:  " + composite_name + ", Area: " + areaid,
              'font': font,
              'font_size': 20, 'height': 10, 'bg': 'white', 'bg_opacity': 127, 'line': 'black'}}]

Now we can either show rhe image directly or save it to disk. When doing this it is possible to also add the decorations (logos and text) and draw the overlays (coastlines, rivers, borders, graticules and points). This is done using the keyword arguments `overlay` and `decorate` in this case. The path to the shape files with coastlines etc need to specified obviously:

In [8]:
local_scn.save_dataset(composite_name,
                       filename='seviri_%s_%s_%s.tif' % (composite_name,
                                                         local_scn.start_time.strftime('%Y%m%d_%H%M'),
                                                         areaid),
                       overlay={'coast_dir': '/home/a000680/data/shapes/',
                                'overlays': {'grid': grid,
                                             'coasts': coast,
                                             'borders': borders,
                                             'rivers': rivers,
                                             'points': points}},
                       decorate={'decorate': decoration})

/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in cos
  return func(*(_execute_task(a, cache) for a in args))
/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in sin
  return func(*(_execute_task(a, cache) for a in args))
/home/a000680/miniconda3/envs/python38/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))
